In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ipynb.fs.full.NBAStatsWebScraper import df_NBA_complete

In [11]:
df_NBA_complete

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2022/23 Salary
0,Precious Achiuwa,C,23,TOR,42,11,22.5,3.7,7.7,0.480,...,1.9,4.4,6.3,1.0,0.6,0.6,1.2,2.0,9.7,2840160
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,...,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6,17926829
2,Bam Adebayo,C,25,MIA,61,61,35.0,8.3,15.5,0.540,...,2.5,7.1,9.6,3.3,1.2,0.8,2.5,2.8,21.2,30351780
3,Ochai Agbaji,SG,22,UTA,43,6,16.6,2.0,4.4,0.444,...,0.6,1.1,1.7,0.7,0.2,0.1,0.4,1.5,5.4,3918360
4,Santiago Aldama,PF,22,MEM,61,18,21.9,3.3,7.0,0.473,...,1.0,3.6,4.5,1.2,0.6,0.7,0.7,2.0,9.2,2094120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,McKinley Wright,PG,24,DAL,20,1,10.3,1.2,2.5,0.469,...,0.3,1.0,1.3,1.9,0.4,0.2,0.6,0.9,2.9,508891
509,Thaddeus Young,PF,34,TOR,52,9,15.1,2.1,3.8,0.551,...,1.3,1.8,3.2,1.4,1.0,0.1,0.8,1.7,4.6,8000000
510,Trae Young,PG,24,ATL,58,58,35.1,8.4,19.6,0.427,...,0.8,2.2,3.0,10.1,1.1,0.2,4.1,1.5,26.6,37096500
511,Cody Zeller,C,30,MIA,7,0,13.7,2.0,3.1,0.636,...,1.6,1.1,2.7,0.7,0.1,0.4,0.7,2.4,6.0,743922
